## The MPArray class

In this section, we exemplify the usage of mpnum in the context of quantum physics. 
The main goal is not to provide a comprehensive introduction, but to showcase the main design choices and goals of mpnum: flexibility, user-friendliness, and expandability.
For a more thorough reference, we refer the reader to the online documentation under [http://mpnum.readthedocs.io/en/latest/](http://mpnum.readthedocs.io/en/latest/).
Let us start by importing the necessary packages.

In [ ]:
import numpy as np
import mpnum as mp

The fundamental data structure of mpnum is the ``MPArray``, which stands for _matrix product array_.
It is composed of an arbitrary number of local tensors with an arbitrary number of legs per site arranged in a linear chain.
MPS and MPOs are special cases of this structure with one and two legs per site, respectively.

We start by performing the TT-SVD from \cref{eq:mps.matrix_product} on a random tensor $X \in \Reals^{2^{10}}$.

In [ ]:
shape = 10 * (2,)
X = np.random.randn(*shape)
X /= np.linalg.norm(X.ravel())
X_mps = mp.MPArray.from_array(X, ndims=1)
X_mps.ndims

This computes the MPS representation of ``X``.
By specifying ``ndim=1``, we make sure the resulting tensor has one leg per site, which we check by `X_mps.ndims`.

Note here and throughout the rest of this section that the internal representation of the local tensors as a list of ``numpy.ndarray`` are hidden from the user behind an acccessible, high-level interface.
However, direct access to the local tensors is provided using the ``MPArray.lt`` property, e.g. to compute how many floating point numbers are used in the MPS representation:

In [ ]:
sum(M.size for M in X_mps.lt)

This is more than twice as large as the number of components for ``X`` itself, which is $2^{10}$, or

In [ ]:
X.size

Since the original tensor $X$ is generated by sampling its components from a normal distribution, it is not compressible in MPS form.
We see that the ranks of the tensor are exponentially increasing towards the middle as expected from \cref{eq:mps.generic_ranks}

In [ ]:
X_mps.ranks

Furthermore, even a moderate compression incurs a large approximation error.

In [ ]:
X_compressed, overlap = X_mps.compression(rank=11)
overlap

In [ ]:
X_compressed.ranks

Let us now demonnstrate the ``MPArray`` class for a compressible state, e.g. the W-state.

In [ ]:
from qutip.states import w_state

psi = w_state(10).data.toarray().reshape((2,) * 10)
psi_mps = mp.MPArray.from_array(psi, ndims=1)
overlap = psi_mps.compress(rank=2)

overlap

In [ ]:
psi_mps.ranks

Note that in contrast to the previous case, we use in-place compression to reduce memory consumption.
Clearly, the rank 2 MPS approximates the W-state up to numerical precission and requires staggeringly fewer parameters.

In [ ]:
sum(M.size for M in psi_mps.lt)

One main motivation behind encapsulating the local tensors in the ``MPArray`` data type is to ensure that they represent a valid MPS at all times and prevent common errors such as mismatch of the virtual dimensions.
Furthermore, it allows us to keep track of the canonical form of the tensor.

In [ ]:
psi_mps.canonical_form

Here, the first number indicates the index up to which all local tensors are left-normalized and the second number the index after which all local tensors are right-normalized.
The ``psi_mps`` tensor in this example is in left-canonical form according to \cref{def:mps.canonical}.
If we change one of the local tensors, e.g. by rescaling, the canonical form changes.

In [ ]:
M = psi_mps.lt[3]
psi_mps.lt.update(3, 2 * M)
psi_mps.canonical_form

To bring it back to full canonical form, we need to call the appropriate method.

In [ ]:
psi_mps.canonicalize(left=9)
psi_mps.canonical_form

## Arithmetic Operations

We now demonstrate the high-level interface for arithmetic operations on ``MPArray`` by simulating the preparation of a $N$-qubit GHZ state
\begin{equation}
  \ket{\GHZ} = \frac{1}{\sqrt 2} \left( \ket{0,\ldots,0} + \ket{1,\ldots,1} \right)
  \label{eq:mpnum.ghz}
\end{equation}
One possible circuit for this task is a succesive application of $\CNOT$ gates~\cite{Nielsen}
\begin{equation}
  \ket{\GHZ} = \CNOT_{N-1,N} \ldots \CNOT_{1,2} H_1 \ket{0,\ldots,0}.
  \label{eq:mpnum.ghz_preparation}
\end{equation}
Here $H_i$ is a Hadamard gate on the $i$-th qubit and $\CNOT_{i,j}$ denotes a controlled not gate with control on qubit $i$ and target on qubit $j$. 
We start by defining the necessary local operations.

In [ ]:
from qutip.qip.gates import hadamard_transform
from qutip.qip.gates import cnot as cnot_transform

hadamard_local = hadamard_transform().data.toarray()
cnot_local = cnot_transform().data.toarray()
cnot_local

Next, generate the initial state in MPS form and convert the local operators to the ``MPArray`` data type.

In [ ]:
N = 10

ket_down_local = np.array([1, 0], dtype=complex)
ket_down = mp.MPArray.from_kron(N * [ket_down_local])
hadamard = mp.MPArray.from_array(hadamard_local, ndims=2)
len(ket_down)

In [ ]:
len(hadamard)

Note that the initial state $\ket{0, \ldots, 0}$ is a product, and hence, can be represented by an MPS of rank 1.

In [ ]:
ket_down.ranks

Since ``cnot_local`` is in matrix form, we cannot directly perform the TT-SVD on it.
First, we have to convert it to a tensor of order four -- two legs per site -- and rearrange the legs in such a way such that legs from the same site are adjacent.

In [ ]:
cnot_local = cnot_local.reshape(4 * (2,)).transpose((0, 2, 1, 3))
cnot =  mp.MPArray.from_array(cnot_local, ndims=2)
len(cnot)

In [ ]:
cnot.ranks

Now we can start to perform the circuit from \cref{eq:mpnum.ghz_preparation}.

In [ ]:
ket_ghz = mp.partialdot(hadamard, ket_down, start_at=0)
ket_ghz.ranks

The function ``partialdot`` performs an efficient contraction of two ``MPArray`` of possibly unequal length.
The result is an ``MPArray`` of the same order and -- since ``hadamard`` is a one-body operator -- of the same rank.
We continue by applying the first CNOT, which results in an MPS with higher rank as CNOT entangles the two qubits on site one and two.

In [ ]:
ket_ghz = mp.partialdot(cnot, ket_ghz, start_at=0)
ket_ghz.ranks

The other CNOT gates follow similarly.

In [ ]:
for site in range(1, N - 1):
    ket_ghz = mp.partialdot(cnot, ket_ghz, start_at=site)
ket_ghz.ranks

This yields an GHZ state in MPS representation.

A different approach is to simply generate the GHZ-state~\eqref{eq:mpnum.ghz} as a diagonal tensor.
The two tensors are equal up to numerical precission.

In [ ]:
ket_ghz2 = mp.diagonal_mpa(np.array([1, 1]), N)
ket_ghz2 /= mp.norm(ket_ghz2)
mp.norm(ket_ghz - ket_ghz2)